In [1]:
import com.toldoven.aoc.notebook.AocClient

val aoc = AocClient.fromEnv().interactiveDay(2018, 10)

aoc.viewPartOne()

--- Day 10: The Stars Align --- 
 It's no use; your navigation system simply isn't capable of providing walking directions in the arctic circle, and certainly not in 1018. 
 The Elves suggest an alternative. In times like these, North Pole rescue operations will arrange points of light in the sky to guide missing Elves back to base. Unfortunately, the message is easy to miss: the points move slowly enough that it takes hours to align them, but have so much momentum that they only stay aligned for a second. If you blink at the wrong time, it might be hours before another message appears. 
 You can see these points of light floating in the distance, and record their position in the sky and their velocity, the relative change in position per second (your puzzle input). The coordinates are all given from your perspective; given enough time, those positions and velocities will move the points into a cohesive message! 
 Rather than wait, you decide to fast-forward the process and calculate what the points will eventually spell. 
 For example, suppose you note the following points: 
 position=< 9, 1> velocity=< 0, 2>
position=< 7, 0> velocity=<-1, 0>
position=< 3, -2> velocity=<-1, 1>
position=< 6, 10> velocity=<-2, -1>
position=< 2, -4> velocity=< 2, 2>
position=<-6, 10> velocity=< 2, -2>
position=< 1, 8> velocity=< 1, -1>
position=< 1, 7> velocity=< 1, 0>
position=<-3, 11> velocity=< 1, -2>
position=< 7, 6> velocity=<-1, -1>
position=<-2, 3> velocity=< 1, 0>
position=<-4, 3> velocity=< 2, 0>
position=<10, -3> velocity=<-1, 1>
position=< 5, 11> velocity=< 1, -2>
position=< 4, 7> velocity=< 0, -1>
position=< 8, -2> velocity=< 0, 1>
position=<15, 0> velocity=<-2, 0>
position=< 1, 6> velocity=< 1, 0>
position=< 8, 9> velocity=< 0, -1>
position=< 3, 3> velocity=<-1, 1>
position=< 0, 5> velocity=< 0, -1>
position=<-2, 2> velocity=< 2, 0>
position=< 5, -2> velocity=< 1, 2>
position=< 1, 4> velocity=< 2, 1>
position=<-2, 7> velocity=< 2, -2>
position=< 3, 6> velocity=<-1, -1>
position=< 5, 0> velocity=< 1, 0>
position=<-6, 0> velocity=< 2, 0>
position=< 5, 9> velocity=< 1, -2>
position=<14, 7> velocity=<-2, 0>
position=<-3, 6> velocity=< 2, -1>
 
 Each line represents one point. Positions are given as <X, Y> pairs: X represents how far left (negative) or right (positive) the point appears, while Y represents how far up (negative) or down (positive) the point appears. 
 At 0 seconds, each point has the position given. Each second, each point's velocity is added to its position. So, a point with velocity <1, -2> is moving to the right, but is moving upward twice as quickly. If this point's initial position were <3, 9> , after 3 seconds, its position would become <6, 3> . 
 Over time, the points listed above would move like this: 
 Initially:
........#.............
................#.....
.........#.#..#.......
......................
#..........#.#.......#
...............#......
....#.................
..#.#....#............
.......#..............
......#...............
...#...#.#...#........
....#..#..#.........#.
.......#..............
...........#..#.......
#...........#.........
...#.......#..........

After 1 second:
......................
......................
..........#....#......
........#.....#.......
..#.........#......#..
......................
......#...............
....##.........#......
......#.#.............
.....##.##..#.........
........#.#...........
........#...#.....#...
..#...........#.......
....#.....#.#.........
......................
......................

After 2 seconds:
......................
......................
......................
..............#.......
....#..#...####..#....
......................
........#....#........
......#.#.............
.......#...#..........
.......#..#..#.#......
....#....#.#..........
.....#...#...##.#.....
........#.............
......................
......................
......................

After 3 seconds:
......................
......................
....................

In [2]:
val input = aoc.input()
input

[position=< 41660,  20869> velocity=<-4, -2>, position=< 41617,  10491> velocity=<-4, -1>, position=< 41669, -10244> velocity=<-4,  1>, position=< 10509, -41348> velocity=<-1,  4>, position=<-41324, -51717> velocity=< 4,  5>, position=< 41616, -30982> velocity=<-4,  3>, position=< 31242, -51716> velocity=<-3,  5>, position=<-20570, -30976> velocity=< 2,  3>, position=< 20882,  51971> velocity=<-2, -5>, position=<-10205, -30981> velocity=< 1,  3>, position=< 31299, -20612> velocity=<-3,  2>, position=<-41320,  51974> velocity=< 4, -5>, position=<-10205, -51723> velocity=< 1,  5>, position=<-10237,  10492> velocity=< 1, -1>, position=<-10181, -10244> velocity=< 1,  1>, position=< 31275, -10243> velocity=<-3,  1>, position=< 52025,  51974> velocity=<-5, -5>, position=< 41634, -20612> velocity=<-4,  2>, position=< 41632, -20610> velocity=<-4,  2>, position=< 52028, -10247> velocity=<-5,  1>, position=<-30972,  51969> velocity=< 3, -5>, position=<-30946, -20609> velocity=< 3,  2>, position=

In [3]:
import utils.geometry.*

data class PositionAndVelocity(
    val initialPosition: Point,
    val velocity: Point,
)

fun PositionAndVelocity.move(): PositionAndVelocity {
    return PositionAndVelocity(this.initialPosition.add(this.velocity), velocity)
}

fun parseInput(input: List<String>): List<PositionAndVelocity> {
    return input.map {
        val regex = Regex("""position=<\s*(-?\d+),\s*(-?\d+)> velocity=<\s*(-?\d+),\s*(-?\d+)>""")
        val (xPos, yPos, xVel, yVel) = regex.find(it)!!.destructured
        PositionAndVelocity(Point(xPos.toInt(), yPos.toInt()), Point(xVel.toInt(), yVel.toInt()))
    }
}
val inputParsed = parseInput(input)
inputParsed 

[PositionAndVelocity(initialPosition=Point(x=41660, y=20869), velocity=Point(x=-4, y=-2)), PositionAndVelocity(initialPosition=Point(x=41617, y=10491), velocity=Point(x=-4, y=-1)), PositionAndVelocity(initialPosition=Point(x=41669, y=-10244), velocity=Point(x=-4, y=1)), PositionAndVelocity(initialPosition=Point(x=10509, y=-41348), velocity=Point(x=-1, y=4)), PositionAndVelocity(initialPosition=Point(x=-41324, y=-51717), velocity=Point(x=4, y=5)), PositionAndVelocity(initialPosition=Point(x=41616, y=-30982), velocity=Point(x=-4, y=3)), PositionAndVelocity(initialPosition=Point(x=31242, y=-51716), velocity=Point(x=-3, y=5)), PositionAndVelocity(initialPosition=Point(x=-20570, y=-30976), velocity=Point(x=2, y=3)), PositionAndVelocity(initialPosition=Point(x=20882, y=51971), velocity=Point(x=-2, y=-5)), PositionAndVelocity(initialPosition=Point(x=-10205, y=-30981), velocity=Point(x=1, y=3)), PositionAndVelocity(initialPosition=Point(x=31299, y=-20612), velocity=Point(x=-3, y=2)), PositionA

In [4]:
val exampleInput = """position=< 9,  1> velocity=< 0,  2>
position=< 7,  0> velocity=<-1,  0>
position=< 3, -2> velocity=<-1,  1>
position=< 6, 10> velocity=<-2, -1>
position=< 2, -4> velocity=< 2,  2>
position=<-6, 10> velocity=< 2, -2>
position=< 1,  8> velocity=< 1, -1>
position=< 1,  7> velocity=< 1,  0>
position=<-3, 11> velocity=< 1, -2>
position=< 7,  6> velocity=<-1, -1>
position=<-2,  3> velocity=< 1,  0>
position=<-4,  3> velocity=< 2,  0>
position=<10, -3> velocity=<-1,  1>
position=< 5, 11> velocity=< 1, -2>
position=< 4,  7> velocity=< 0, -1>
position=< 8, -2> velocity=< 0,  1>
position=<15,  0> velocity=<-2,  0>
position=< 1,  6> velocity=< 1,  0>
position=< 8,  9> velocity=< 0, -1>
position=< 3,  3> velocity=<-1,  1>
position=< 0,  5> velocity=< 0, -1>
position=<-2,  2> velocity=< 2,  0>
position=< 5, -2> velocity=< 1,  2>
position=< 1,  4> velocity=< 2,  1>
position=<-2,  7> velocity=< 2, -2>
position=< 3,  6> velocity=<-1, -1>
position=< 5,  0> velocity=< 1,  0>
position=<-6,  0> velocity=< 2,  0>
position=< 5,  9> velocity=< 1, -2>
position=<14,  7> velocity=<-2,  0>
position=<-3,  6> velocity=< 2, -1>""".split("\n")
val exampleInputParsed = parseInput(exampleInput)
exampleInputParsed

[PositionAndVelocity(initialPosition=Point(x=9, y=1), velocity=Point(x=0, y=2)), PositionAndVelocity(initialPosition=Point(x=7, y=0), velocity=Point(x=-1, y=0)), PositionAndVelocity(initialPosition=Point(x=3, y=-2), velocity=Point(x=-1, y=1)), PositionAndVelocity(initialPosition=Point(x=6, y=10), velocity=Point(x=-2, y=-1)), PositionAndVelocity(initialPosition=Point(x=2, y=-4), velocity=Point(x=2, y=2)), PositionAndVelocity(initialPosition=Point(x=-6, y=10), velocity=Point(x=2, y=-2)), PositionAndVelocity(initialPosition=Point(x=1, y=8), velocity=Point(x=1, y=-1)), PositionAndVelocity(initialPosition=Point(x=1, y=7), velocity=Point(x=1, y=0)), PositionAndVelocity(initialPosition=Point(x=-3, y=11), velocity=Point(x=1, y=-2)), PositionAndVelocity(initialPosition=Point(x=7, y=6), velocity=Point(x=-1, y=-1)), PositionAndVelocity(initialPosition=Point(x=-2, y=3), velocity=Point(x=1, y=0)), PositionAndVelocity(initialPosition=Point(x=-4, y=3), velocity=Point(x=2, y=0)), PositionAndVelocity(i

In [8]:
var currentPoints = inputParsed
repeat(1000000) {
    val grid = currentPoints.map { it.initialPosition }.associateWith { '#' }
    val minX = grid.filter { it.key.x > 0 }.minBy { it.key.x }.key.x
    val minY = grid.filter { it.key.y > 0 }.minBy { it.key.y }.key.y
    val maxX = grid.maxBy { it.key.x }.key.x
    val maxY = grid.maxBy { it.key.y }.key.y
    
    if (abs(minY - maxY) < 100 && abs(minX - maxX) < 100) {
        println()
        println("Iteration num $it")
        grid.printGrid('.')
    }
    currentPoints = currentPoints.map {
        it.move()
    }
}


Iteration num 10366

..............#....................#........#..#..........#......#..........................
.....................................................................#......................
........................#.................................................#.................
........................#........#.......#....#...#......#.#...#.....##.........##...#......
.........................#..................#.......##..............#.......................
......................#.................#............#........#.............................
..............#........##.....#............#......#.........#.#.............#...............
#............#.............#...#......#.#......#......#....#..#..................#.#........
..........................#.#......#...........#...#..#..............................#......
.............#......#..#..#.#......#.#.#..#...#..##..#..#.......##..##...#.##...............
.........#.....#...................#....#...#..#

In [6]:
aoc.submitPartOne("KBJHEZCB")

Your answer: KBJHEZCB. 
 That's the right answer! You are one gold star closer to fixing the time stream. [Continue to Part Two]

In [7]:
aoc.viewPartTwo()

--- Part Two --- 
 Good thing you didn't have to wait, because that would have taken a long time - much longer than the 3 seconds in the example above. 
 Impressed by your sub-hour communication capabilities, the Elves are curious: exactly how many seconds would they have needed to wait for that message to appear?

In [9]:
aoc.submitPartTwo("10369")

Your answer: 10369. 
 That's the right answer! You are one gold star closer to fixing the time stream. 
 You have completed Day 10! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .